# 3D Visualizer

This notebook will show you how to create a 3D model of a specific region (e.g. tumor) in an MRI volume.

In [36]:
import SimpleITK as sitk
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
import numpy as np
from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

Using the SimpleITK library, we can read the image from the mha file and then convert that into a 3D numpy array.

In [37]:
path = "path/to/file.mha" # insert the path to your data here

# Create a numpy array of the tumor
tumor = sitk.ReadImage(path, imageIO="MetaImageIO")
tumor = sitk.GetArrayFromImage(tumor)

The array we just created is full of numbers representing different labels. 0 indicates nothing and 1 is for tumor. For this example, we only want to model the tumor region, so any other numbers should be changed to 0. 

In [38]:
# Get only the tumor region (represented by the number 1)
tumor =  np.where(tumor != 1, 0, 1)

The SimpleITK method GetArrayFromImage returns a numpy array in the form DHW (depth, height, width). But the marching cubes method we will be using later requires the array be in the form WHD. So we will fix that here. Swapping the axis rotated the array, so the next line simply undoes the rotation.

In [39]:
tumor = np.swapaxes(tumor, 0, 2)
tumor = np.rot90(tumor, 3)

We will use the marching cubes algorithm from the skimage library to get the vertices and faces from our tumor array. From there, we can take these outputs and generate a 3D model using plotly.

In [51]:
# Use the marching cubes algorithm to generate a 3D mesh
verts, faces, normals, values = measure.marching_cubes(tumor, 0)

x_coords = verts[:, 0]
y_coords = verts[:, 1]
z_coords = verts[:, 2]

x_face_index = faces[:, 0]
y_face_index = faces[:, 1]
z_face_index = faces[:, 2]

# Create the plotly mesh
plotly_mesh = go.Mesh3d(
    x=x_coords,
    y=y_coords,
    z=z_coords,
    i=x_face_index,
    j=y_face_index,
    k=z_face_index,
    color="red" # Change the color to whatever you like!
    )

# Show the model
fig = go.Figure(data=plotly_mesh)
fig.update_layout(title="3D Tumor Model", width=500, height=500)
fig.show()

You can use the code below to save the generated plotly model to an HTML file which will keeo the interactive component. With the HTML file, you can open it in your browser and embed it into a website.

In [49]:
# Save plotly figure to html file
fig.write_html("saved_plots/example_plot.html")